In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### Importazione dataframe

In [2]:
df1 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_23mar_to_29mar.csv')
df2 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_30mar_to_4apr.csv')

df1["measured_at"] = pd.to_datetime( df1["measured_at"] )
df2["measured_at"] = pd.to_datetime( df2["measured_at"] )

mask_30mar = np.asarray( df1[ df1['measured_at'].dt.day == 30].index )
df1.drop( mask_30mar, inplace = True )

mask_29mar = np.asarray( df2[ df2['measured_at'].dt.day == 29].index )
df2.drop( mask_29mar, inplace = True )

mask_5apr = np.asarray( df2[ df2['measured_at'].dt.day == 5].index )
df2.drop( mask_5apr, inplace = True )

df = pd.concat([df1,df2])

In [3]:
df.drop(['id', 'tipologia_lavorazione', 'cicli_da_eseguire',
        'cicli_eseguiti', 'tempo_miscelazione_eseguito',
        'stato_tramoggia', 'tara_cono', 'lotto',
        'created_at', 'updated_at'], axis=1, inplace=True)

In [4]:
mask_no_cono = np.asarray( df[ df['id_cono'] > 24 ].index )
df.drop( mask_no_cono, inplace = True )

In [5]:
df["measured_at"] = pd.to_datetime( df["measured_at"] )

### Preparazione dati di supporto

In [6]:
buffer = [n for n in range(21, 29)]
buffer.extend([n for n in range(31, 39)])

legenda_gualchierani = {
    'fuori_impianto': [0],
    'ing_zona_misc': [1],
    'catenaria_ing_misc': [45, 46],
    'navetta_serv_misc': [43],
    'pesatura': [41, 42],
    'miscelatore': [2, 8],
    'cat_usc_pieni': [3],
    'nav_serv_carr_pieni': [4],
    'nav_serv_carr_vuoti': [5],
    'buffer': buffer,
    'carroponte_1': [6],
    'carroponte_2': [7],
    'estrusore': [n for n in range(11, 19)],
    'handling_post_est': [6, 5, 4],
    'catenaria_vuoti': [47, 48],
    'catenaria_uscita_vuoti': [49],
    'pinza_vuoti': [50],
    'catenaria_parcheggi': [n for n in range(51, 58)],
    'nav_serv_parcheggi': [44],
    'manuale': [68, 70]
}

dict_states = {
    0: 'Not_def',
    1: 'V_per_park',
    2: 'P_da_misc',
    3: 'P_da_est',
    4: 'P_remove',
    5: 'P_recheck',
    6: 'P_per_park',
    7: 'V_da_pes',
    8: 'V_remove',
    9: 'V_lavaggio'
}

starter = ['V_da_pes/nav_serv_carr_pieni', 'V_da_pes/nav_serv_carr_vuoti',
'V_da_pes/catenaria_vuoti', 'V_da_pes/catenaria_parcheggi',
'V_da_pes/nav_serv_parcheggi', 'V_da_pes/catenaria_ing_misc',
'V_da_pes/ing_zona_misc']

finisher = ['P_da_est/estrusore', 'Not_def/fuori_impianto']

finisher2 = ['V_per_park/estrusore', 'V_da_pes/estrusore', 'Not_def/fuori_impianto']

### Preparazione funzioni

In [7]:
#  questa funzione estrae dal df il primo record in ordine temporale per ogni ubicazione

def create_df_t_new(df, cono_in_esame):
    mask = df['id_cono'] == cono_in_esame
    df1 = df[mask]

    ubicazione = []
    stato = []
    timestamp = []
    for i in range(len(df1) - 1):
        if i == 0:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])
        if df1.iloc[i, 1] in legenda_gualchierani['estrusore']:
            if ((df1.iloc[i, 1] == df1.iloc[i - 1, 1] and df1.iloc[i, 1] != df1.iloc[i + 1, 1])
                    or df1.iloc[i, 1] != df1.iloc[i - 1, 1]):
                ubicazione.append(df1.iloc[i, 1])
                stato.append(df1.iloc[i, 2])
                timestamp.append(df1.iloc[i, 8])
        elif df1.iloc[i, 1] == df1.iloc[i - 1, 1]:
            continue
        else:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])

#  estrae i valori dalla memoria del PLC li traduco in parole
    for i in range(len(ubicazione)):
        for k,v in legenda_gualchierani.items():
            if ubicazione[i] in v:
                ubicazione[i] = k

    for i in range(len(stato)):
        stato[i] = dict_states[stato[i]]

#  crea il dataframe che sarà input alla funzione di individuazione della tracce
    df_t = pd.DataFrame(list(zip(stato, ubicazione, timestamp)), columns =['stato', 'ubicazione', 'timestamp'])
    df_t.sort_values('timestamp', inplace=True)
    df_t['ID'] = 0
    df_t['CRC'] = 0
    df_t['rif'] = (df_t.stato.astype('str') + '/' + df_t.ubicazione.astype('str'))
    df_t.drop_duplicates(inplace=True)
    return(df_t)

In [8]:
def create_traces_new(df_t, cono_in_esame, finisher):
    global trace
    df_t2 = df_t.reset_index(drop=True).copy()
    cont_related_counter = 0
    bol_end = False
    bol_start = False
    for idx in df_t2.index[1:]:
        global trace
        if df_t2.loc[idx, 'stato'] == 'V_da_pes' and df_t2.loc[idx - 1, 'stato'] != 'V_da_pes' and not bol_start:
            bol_end = False
            bol_start = True
            trace += 1
            cont_related_counter += 1
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
        elif df_t2.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
            bol_end = True
            bol_start = False
        elif not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
    
    mask_no_trace = np.asarray( df_t2[ df_t2['ID'] == 0 ].index )
    df_t2.drop( mask_no_trace, inplace = True )
    
    #  qualche trick per limitare i doppioni
    trace = trace + 10
    df_t2['ID_unique'] = (df_t2.CRC.astype('str') + 'C' + str(cono_in_esame) + df_t2.ID.astype('str'))
    if df_t2['CRC'].isnull().values.any():
        print('CI SONO DEI NULL SECONDA FUNZIONE  CONO  ', cono_in_esame)
    return(df_t2)

### Calcolo tracce

In [9]:
trace = 0
for c in range(1, 25):
    global trace
    start = datetime.now()
    df_pre_trace = create_df_t_new(df, c)
    if c == 1:
        df_traces = create_traces_new(df_pre_trace, c, finisher2)
    else:
        df_traces = pd.concat([df_traces, create_traces_new(df_pre_trace, c, finisher2)])
    end = datetime.now()
    print('TEMPO ELABORAZIONE CONO {} {}'.format(c, end-start))

TEMPO ELABORAZIONE CONO 1 0:00:07.840144
TEMPO ELABORAZIONE CONO 2 0:00:05.049628
TEMPO ELABORAZIONE CONO 3 0:00:08.496981
TEMPO ELABORAZIONE CONO 4 0:00:08.817412
TEMPO ELABORAZIONE CONO 5 0:00:07.104648
TEMPO ELABORAZIONE CONO 6 0:00:07.030327
TEMPO ELABORAZIONE CONO 7 0:00:07.558540
TEMPO ELABORAZIONE CONO 8 0:00:13.356991
TEMPO ELABORAZIONE CONO 9 0:00:07.786431
TEMPO ELABORAZIONE CONO 10 0:00:11.727906
TEMPO ELABORAZIONE CONO 11 0:00:08.483467
TEMPO ELABORAZIONE CONO 12 0:00:08.044947
TEMPO ELABORAZIONE CONO 13 0:00:16.899730
TEMPO ELABORAZIONE CONO 14 0:00:08.241729
TEMPO ELABORAZIONE CONO 15 0:00:07.335373
TEMPO ELABORAZIONE CONO 16 0:00:07.608416
TEMPO ELABORAZIONE CONO 17 0:00:11.052048
TEMPO ELABORAZIONE CONO 18 0:00:09.383132
TEMPO ELABORAZIONE CONO 19 0:00:05.975189
TEMPO ELABORAZIONE CONO 20 0:00:05.915307
TEMPO ELABORAZIONE CONO 21 0:00:07.622202
TEMPO ELABORAZIONE CONO 22 0:00:08.554674
TEMPO ELABORAZIONE CONO 23 0:00:10.231083
TEMPO ELABORAZIONE CONO 24 0:00:08.067631


In [10]:
df_traces.to_csv('C:/Users/HP/Desktop/df_traces_end_pes.csv')

In [11]:
len(df_traces['ID_unique'].unique())

1219

In [12]:
df_traces

,stato,ubicazione,timestamp,ID,CRC,rif,ID_unique
14,V_da_pes,catenaria_ing_misc,2022-03-23 01:35:26,1,1,V_da_pes/catenaria_ing_misc,1C11
15,V_da_pes,catenaria_ing_misc,2022-03-23 01:40:00,1,1,V_da_pes/catenaria_ing_misc,1C11
16,V_da_pes,navetta_serv_misc,2022-03-23 01:54:18,1,1,V_da_pes/navetta_serv_misc,1C11
17,V_da_pes,pesatura,2022-03-23 01:54:52,1,1,V_da_pes/pesatura,1C11
33,V_da_pes,nav_serv_parcheggi,2022-03-23 04:20:09,2,2,V_da_pes/nav_serv_parcheggi,2C12
...,...,...,...,...,...,...,...
2192,V_da_pes,navetta_serv_misc,2022-04-04 21:07:44,1427,132,V_da_pes/navetta_serv_misc,132C231427
2193,V_da_pes,pesatura,2022-04-04 21:08:18,1427,132,V_da_pes/pesatura,132C231427
2209,V_da_pes,nav_serv_parcheggi,2022-04-04 22:51:50,1428,133,V_da_pes/nav_serv_parcheggi,133C231428
2210,V_da_pes,catenaria_ing_misc,2022-04-04 22:52:24,1428,133,V_da_pes/catenaria_ing_misc,133C231428
